In [1]:
import pandas as pd

In [2]:
df = pd.read_json("meusfilmes.json")
# Tive que fazer o download do arquivo JSON enviando antes, pois no momento da criação da trusted, a api não estava funcionando.

In [3]:
# Aqui peguei todo o dataframe e onde tinha dados "None", transformei em "Null" ou "0".
df['backdrop_path'].fillna("Null", inplace=True)
df['genre_ids'].fillna(1, inplace=True)
df['id'].fillna(0, inplace=True)
df['original_language'].fillna('Null', inplace=True)
df['original_title'].fillna('Null', inplace=True)
df['overview'].fillna('Null', inplace=True)
df['popularity'].fillna(0, inplace=True)
df['poster_path'].fillna("Null", inplace=True)
df['release_date'].fillna("0-0-0", inplace=True)
df['title'].fillna("Null", inplace=True)
df['video'].fillna(False, inplace=True)
df['vote_average'].fillna(0, inplace=True)
df['vote_count'].fillna(0, inplace=True)


In [4]:
# Os dados estão no formato correto... obs:Object=String
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              97 non-null     bool   
 1   backdrop_path      97 non-null     object 
 2   genre_ids          97 non-null     object 
 3   id                 97 non-null     int64  
 4   original_language  97 non-null     object 
 5   original_title     97 non-null     object 
 6   overview           97 non-null     object 
 7   popularity         97 non-null     float64
 8   poster_path        97 non-null     object 
 9   release_date       97 non-null     object 
 10  title              97 non-null     object 
 11  video              97 non-null     bool   
 12  vote_average       97 non-null     float64
 13  vote_count         97 non-null     int64  
 14  Orçamento          97 non-null     int64  
 15  Receita            97 non-null     int64  
 16  Duração            97 non-nu

In [5]:
nomes_colunas = ['Adulto', 'Pano_Fundo', 'Id_Genero', 'Id', 'Linguagem_Original', 'Titulo_Original', 'Visao_Geral', 'Popularidade', 'Poster', 'Data_Lancamento', 'Titulo', 'Video', 'Votacao', 'Votos', 'Orcamento', 'Receita', 'Duracao']

In [6]:
df.columns = nomes_colunas

In [7]:
# Minha escolha de filmes foi mais específica, então em relação a tratamento é basicamente isso. Acredito que os dados estão confiaveis.
# O conceito de RAW / Trusted / Refined consigo entender.
# Devido ao meu atraso resolvi fazer com pandas por ser mais rápido, para tentar acompanhar a sprint;


## Começando a usar o pyspark para transformar o json limpo em parquet

In [8]:
import findspark 
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql import functions as Func

In [9]:
df_spark = spark.createDataFrame(df)
#criando um dataframe spark

In [10]:
df_spark.show(100)

+------+--------------------+--------------------+-------+------------------+--------------------+--------------------+------------------+--------------------+---------------+--------------------+-----+------------------+-----+---------+----------+-------+
|Adulto|          Pano_Fundo|           Id_Genero|     Id|Linguagem_Original|     Titulo_Original|         Visao_Geral|      Popularidade|              Poster|Data_Lancamento|              Titulo|Video|           Votacao|Votos|Orcamento|   Receita|Duracao|
+------+--------------------+--------------------+-------+------------------+--------------------+--------------------+------------------+--------------------+---------------+--------------------+-----+------------------+-----+---------+----------+-------+
| false|/feSiISwgEpVzR1v3...|       [28, 12, 878]| 609681|                en|         The Marvels|Carol Danvers, ak...|           732.498|/Ag3D9qXjhJ2FUkrl...|     2023-11-08|         The Marvels|false|               6.5|  387|27

# Falta salvar esse DATAFRAME SPARK COMO PARQUET E ENVIAR PRO S3

In [11]:
df_spark.write.format("parquet").save("C:\\Users\\victo\\Downloads\\DATA E ANALYTICS\\Documentos\\Sprint_9\\parquet")
# Salvando o dataframe spark como parquet.. tentei salvar o dataframe inteiro em um único arquivo, sem sucesso.

In [17]:
testando = spark.read.format("parquet").load("C:\\Users\\victo\\Downloads\\DATA E ANALYTICS\\Documentos\\Sprint_8\\parquet\\part-00000-7f53a870-5f44-471d-9c62-b464e6c4ff9d-c000.snappy.parquet")
#um teste pra saber se os dados estavam certos...

In [12]:
df_spark.write.saveAsTable("df_trusted")
#então salvei o dataframe em uma tabela, para *tentar* salvar em um unico arquivo parquet. Também não consegui.

In [13]:
df_spark = spark.sql("select * from df_trusted")

In [44]:
df_spark.write.parquet("C:\\Users\\victo\\Downloads\\DATA E ANALYTICS\\Documentos\\Sprint_9\\parquet_certo")
#Salvando como parquet... e aqui mesmo irei mandar para o S3

In [49]:
import boto3
import os 

client = boto3.client('s3',
                      aws_access_key_id="*******",
                      aws_secret_access_key="*****"
                      )

client.upload_file("parquet.parquet", "pimprawzone","TRT/Movies/Parquet/arquivo.parquet")
# Busquei como enviar uma pasta ou vários arquivos de um só vez para o S3, mas não achei info. Como não queria enviar arquivo por arquivo, precisei ir pro AWS Glue.

In [ ]:
# sinto muito por não conseguir concluir no prazo e da maneira que deveria ser feita... essa foi a forma que consegui seguir para pelo menos concluir.

In [15]:
df_spark.show(5)

+------+--------------------+-------------+------+------------------+--------------------+--------------------+------------+--------------------+---------------+--------------------+-----+------------------+-----+---------+---------+-------+
|Adulto|          Pano_Fundo|    Id_Genero|    Id|Linguagem_Original|     Titulo_Original|         Visao_Geral|Popularidade|              Poster|Data_Lancamento|              Titulo|Video|           Votacao|Votos|Orcamento|  Receita|Duracao|
+------+--------------------+-------------+------+------------------+--------------------+--------------------+------------+--------------------+---------------+--------------------+-----+------------------+-----+---------+---------+-------+
| false|/4NWWpT0jiMUak8r6...|[28, 12, 878]|  1771|                en|Captain America: ...|During World War ...|      56.576|/vSNxAJTlD0r02V9s...|     2011-07-22|Captain America: ...|false|             6.995|20383|140000000|370569774|    124|
| false|/iYdgEUE2W2aJkgqf...|[28

A CONCLUSÃO FOI FEITA NO AWS GLUE OU NO ARQUIVO, 'DESAFIO-TRUSTED_GLUE'.